# NPZD slab model

For the first model example I will need:
- Forcing input
- Forcing_fluxes:
    - mixing
    - sinking
- 4 components (single) N, P, Z, D
- Fluxes:
TODO: GO OVER THE equations! Base model structure in math!
    - Grazing - linked to all NPZD
    - PhytoplanktonGrowth
        - Light - Dependence
        - Nutrients - Dependence
        - Temp - Dependence
    - mortality
    - remineralisation
    - higher order pred, etc

In [1]:
import numpy as np
import xsimlab as xs

In [2]:
import os,sys,inspect
sys.path.insert(1, os.path.join(sys.path[0], '..'))

import phydra


from phydra.utility.xsimlabwrappers import phydra_setup

In [3]:
from phydra.processes.main import Grid0D, Boundary0D
from phydra.processes.components import Component, Time, make_Component
from phydra.processes.environments import BaseEnvironment, Slab
from phydra.processes.fluxes import (BaseFlux, LimitedGrowth, 
                                     LinearMortality, GrazingFlux, 
                                     GrazingMultiFlux, make_flux,
                                    make_multiflux)
from phydra.processes.forcingfluxes import Mixing, Sinking, Upwelling, make_FX_flux
from phydra.processes.forcing import SinusoidalMLD, ConstantN0
from phydra.processes.gekkocontext import GekkoContext, GekkoSolve, InheritGekkoContext

In [4]:
from phydra.processes.forcing import NutrientForcing, MLDForcing

In [5]:
print('xarray-simlab version: ', xs.__version__)
print('phydra version: ', phydra.__version__)

xarray-simlab version:  0.4.1
phydra version:  0+untagged.75.g44d95e3.dirty


In [6]:
model = xs.Model({
    'time':Time, 'grid':Grid0D, 'ibcs':Boundary0D, 
    
    'context':GekkoContext,
                  
    'N':make_Component('Nutrient','N'), 'P':make_Component('Phytoplankton','P'),
    'Z':make_Component('Zooplankton','Z'), 'D':make_Component('Detritus','D'),
    'Env':Slab, 
    
    # Fluxes
    'P_Growth':make_flux(LimitedGrowth, 'G'), 
    'P_LinMortality':make_flux(LinearMortality, 'M'), 
    
    'Grazing':make_multiflux(GrazingMultiFlux, 'X'), 
    'Z_LinMortality':make_flux(LinearMortality, 'M'),
    
    
    # ForcingFluxes
    'Sinking':make_FX_flux(Sinking, 'S'),
    'Upwelling':make_FX_flux(Upwelling, 'U'),
    
    'MLD':SinusoidalMLD, 
    'N0':ConstantN0, 
    
    'solver':GekkoSolve
})

model

('XR',)
('XC',)


<xsimlab.Model (18 processes, 31 inputs)>
context
time
    days               [in] ('time',) time in days
grid
    shape              [in]
    length             [in]
ibcs
    ibc                [in] Initial Boundary Conditions
N
    init               [in]
    dim                [in]
P
    dim                [in]
    init               [in]
Z
    init               [in]
    dim                [in]
D
    init               [in]
    dim                [in]
Z_LinMortality
    R_label            [in] label of component that experiences mor...
    C_label            [in] label of component that mortality feeds...
    mortality_rate     [in] mortality rate of component
N0
    constant_N0        [in] value of the constant Nutrient forcing
MLD
Upwelling
    kappa              [in] constant mixing coefficient
    C_labels           [in] u label of component(s) that grows
P_LinMortality
    R_label            [in] label of component that experiences mor...
    C_label            [in] label of c

In [7]:
model.index_vars 


[('time', 'time'),
 ('N', 'N'),
 ('P', 'P'),
 ('Z', 'Z'),
 ('D', 'D'),
 ('Env', 'components'),
 ('Env', 'fluxes'),
 ('Env', 'forcingfluxes'),
 ('P_Growth', 'G'),
 ('P_LinMortality', 'M'),
 ('Grazing', 'X'),
 ('Z_LinMortality', 'M'),
 ('Sinking', 'S'),
 ('Upwelling', 'U')]

In [8]:
in_ds = phydra_setup(
    model=model,
    input_vars={
        'time__days': ('time', np.arange(0, 365*3)), #365*
        
        'N':{'dim':1,'init':1}, 
        'P':{'dim':1,'init':.1},
        'Z':{'dim':1,'init':0.1},
        'D':{'dim':1,'init':0.1},
        
        'P_Growth':{'mu':1.,
               'R_label':'N',
               'C_label':'P', 
               'halfsat':0.5},
        
        'P_LinMortality':{'mortality_rate':0.1,
               'R_label':'P',
               'C_label':'D'},
        
        'Grazing':{'grazing_rate':0.1,
               'halfsat':0.5,
               'R_labels':['P','D'],
               'C_labels':['Z']},
        'Z_LinMortality':{'mortality_rate':0.1,
               'R_label':'Z',
               'C_label':'D'},
        
        'Sinking':{'kappa':0.1,
               'C_labels':['D','P']},
        
        'Upwelling':{'kappa':0.1,
               'C_labels':['N']},
        
        'N0__constant_N0':2.
    },
    output_vars={
        'Env__comp_output':None,
        'Env__flux_output':None,
        'Env__fxflux_output':None
    }
                 )

TypeError: Could not get dimension labels from model for variable 'Grazing__R_labels' with value ['P', 'D']

In [9]:
with model:
    out_ds = in_ds.xsimlab.run()

NameError: name 'in_ds' is not defined

In [ ]:
out_ds

In [ ]:
out_ds.Env__comp_output.plot.line(x='time')

In [ ]:
out_ds.Env__flux_output.plot.line(x='time')

In [ ]:
out_ds.Env__fxflux_output.plot.line(x='time')